In [1]:
import networkx as nx
from collections import defaultdict

In [2]:
def build_test_graph():
    G = nx.Graph()
    G.add_edge(1, 2, weight=2)  # specify edge data
    G.add_edge(3, 4, weight=4)  # specify edge data
    G.add_edge(1, 4, weight=3)  # specify edge data
    G.add_edge(2, 4, weight=3)  # specify edge data
    G.add_edge(1, 3, weight=5)  # specify edge data
    return G

In [10]:
def _has_MST(G, start, end, visited, threshold, debug = False):
    visited.add(start)
    if debug:
        print(start)
    adj = dict(G.adj[start])
    for n in adj:
        if n in visited or adj[n]["weight"] >= threshold:
            continue
        if n == end:
            return False
        if not _has_MST(G, n, end, visited, threshold):
            return False
    return True        

def has_MST(G, edge, debug = False):
    u,v,w = edge
    w = w["weight"]
    return _has_MST(G, u, v, set(), w)

In [11]:
from networkx.utils import UnionFind
from math import isnan

def boruvka_mst_edges(G, fixed, debug = False):
    """Iterate over edges of a Borůvka's algorithm min/max spanning tree.

    Parameters
    ----------
    G : NetworkX Graph
        The edges of `G` must have distinct weights,
        otherwise the edges may not form a tree.

    """
    # Initialize a forest, assuming initially that it is the discrete
    # partition of the nodes of the graph.
    forest = UnionFind(G)
    
    # Add u and v to the same forest
    yield fixed[0], fixed[1], fixed[2]
    forest.union(fixed[0], fixed[1])
    
    if debug:
        for component in forest.to_sets():
            print(component)

    def best_edge(component):
        """Returns the optimum (minimum) edge on the edge
        boundary of the given set of nodes.

        A return value of ``None`` indicates an empty boundary.
        """
        minwt = float('inf')
        boundary = None
        for e in nx.edge_boundary(G, component, data=True):
            wt = e[-1]["weight"]
            if isnan(wt):
                msg = "NaN found as an edge weight. Edge %s"
                raise ValueError(msg % (e,))
            if wt < minwt:
                minwt = wt
                boundary = e
        return boundary

    # Determine the optimum edge in the edge boundary of each component
    # in the forest.
    best_edges = (best_edge(component) for component in forest.to_sets())
    best_edges = [edge for edge in best_edges if edge is not None]

    # If each entry was ``None``, that means the graph was disconnected,
    # so we are done generating the forest.
    while best_edges:
        # Determine the optimum edge in the edge boundary of each
        # component in the forest.
        #
        # This must be a sequence, not an iterator. In this list, the
        # same edge may appear twice, in different orientations (but
        # that's okay, since a union operation will be called on the
        # endpoints the first time it is seen, but not the second time).
        #
        # Any ``None`` indicates that the edge boundary for that
        # component was empty, so that part of the forest has been
        # completed.
        best_edges = (best_edge(component) for component in forest.to_sets())
        best_edges = [edge for edge in best_edges if edge is not None]
        # Join trees in the forest using the best edges, and yield that
        # edge, since it is part of the spanning tree.
        for u, v, d in best_edges:
            if forest[u] != forest[v]:
                yield u, v, d
                forest.union(u, v)

In [12]:
def print_edges(edges):
    weight = 0
    print("Edges of MST:")
    for e in edges:
        w = e[2]["weight"]
        print(e[0], "-->", e[1], "(", w ,")")
        weight += w
    print("Total weight: ", weight)


In [15]:
G = build_test_graph()
fixed = [4,3,{"weight": 4}]
if has_MST(G, fixed):
    print_edges(boruvka_mst_edges(G, fixed))

Edges of MST:
4 --> 3 ( 4 )
1 --> 2 ( 2 )
4 --> 1 ( 3 )
Total weight:  9
